In [1]:
#import libraries
import numpy as np
import pandas as pd
import os

In [2]:
# join the raw file folder
data_dir = os.path.join(os.getcwd(), "parquet_files")

In [3]:
# load the file
def load_data(filename):
  return pd.read_parquet(filename)

# create volume moving average
def create_volume_moving_average(df, candles = 8):
  df[f"vma_{candles}"] = df["volume"].rolling(window=candles).mean()
  df.dropna()
  return df

# resample logic
def resample_ohlcv(df, timeframe):
  return (df.resample(timeframe, label="right", closed="right").agg({
            "open": "first",
            "high": "max",
            "low": "min",
            "close": "last",
            "volume": "sum",
            "open_interest": "last"
        })
        .dropna(subset=["open"]))

# candle colour logic
def candlestick_colours(df):
  df["candle"] = np.where(
    df["open"]>df["close"],"red", 
    np.where(df["open"]==df["close"],"white","green")
  )
  return df

# reversal logic
def reversal(df):
  df["reversal"] = "None"
  bullish =( df["candle"].shift(1) == "red") & (df["candle"]=="green")
  bearish = (df["candle"].shift(1) == "green") & (df["candle"]=="red")
  df.loc[bullish, "reversal"] = "bullish"
  df.loc[bearish, "reversal"] = "bearish"
  return df

# englufing logic
def engulfing(df):
    df["engulfing"] = "None"
    bullish = (
        (df["reversal"] == "bullish") &
        (df["close"] > df["open"].shift(1)) &
        (df["open"] < df["close"].shift(1))
    )
    bearish = (
        (df["reversal"] == "bearish") &
        (df["close"] < df["open"].shift(1)) &
        (df["open"] > df["close"].shift(1))
    )
    df.loc[bullish, "engulfing"] = "bullish"
    df.loc[bearish, "engulfing"] = "bearish"
    return df

# volume logic
def volume_check(df, candles=8):
    df["volume_check"] = "lower"
    df.loc[df["volume"] > df[f"vma_{candles}"], "volume_check"] = "higher"
    return df

  
  
    
  
  

In [4]:
first_test = os.path.join(data_dir, "RELIANCE.parquet")
reliance = load_data(first_test)
reliance = resample_ohlcv(reliance, "5min")
reliance = create_volume_moving_average(reliance, 8)
reliance = candlestick_colours(reliance)
relinace = reversal(reliance) 
relinace = engulfing(reliance)
relinace = volume_check(reliance, 8)